In [1]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

from datetime import datetime

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

from docx2pdf import convert
import os

In [2]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

In [3]:
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [7]:
fecha = '2023-05-08'

In [12]:
query_dia = {'query': 'select * from dataset_291595 where date(fecha_registro) = \'{fech}\''.format(fech=fecha)}
project = 'https://app.amigocloud.com/api/v1/projects/31805/sql'
select_dia = amigocloud.get(project, query_dia)
data_dia = select_dia['data'][]
#data_dia['fecha_registro'] = datetime.strptime(data_dia['fecha_registro'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
#data_dia

In [43]:
id_insp = 2
# DATOS DE LAS INPECCION
query = {'query': 'select * from dataset_291595 where id={id}'.format(id=id_insp)}
project = 'https://app.amigocloud.com/api/v1/projects/31805/sql'
select = amigocloud.get(project, query)
data = select['data'][0]

# convertir fecha a formato dd/mm/aaaa
data['fecha_registro'] = datetime.strptime(data['fecha_registro'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
insp = collections.namedtuple("insp", data.keys())(*data.values())

In [44]:
insp

insp(fecha_registro='09/05/2023', amigo_id='9aa11d40fb3d41e1be7b60054ef10c77', ubicacion='0101000020E61000008F37F92D3A954FC066D2F077394631C0', canhero='388 / AGROPECUARIA MARIANA S.R.L.', propiedad='19 / SANTA BARBARA', lote='L28.1', area=12.7331063169, textura='FY', variedad='SP 83-5073', repeticiones=None, responsable='Juan Pablo Jaldin', observaciones='El operador está dando un mal uso del desputador y corte de base debido a la velocidad alta está dejando los tocones astillados se va a orientar al operador ', id=2, tipo_cosecha='MECANISADA')

In [45]:
# DATOS DE MUESTRAS
query = {'query': 'select * from dataset_291604 where calidad_ref_muestra=\'{amigo_id}\''.format(amigo_id=insp.amigo_id)}
project = 'https://app.amigocloud.com/api/v1/projects/31805/sql'
select = amigocloud.get(project, query)
data = select['data']

In [46]:
muestras =[]
for i in data:
    muestra = collections.namedtuple("muestra", i.keys())(*i.values())
    muestras.append(muestra)

In [47]:
muestras

[muestra(fecha_registro='2023-05-09 19:57:41+00:00', amigo_id='49a7b4ac30ad45c3b982cc4dfa245f17', punto='0101000020E6100000D698C6E52E954FC038899C19584631C0', nombre_muestra='M1', astillas=0.0, tocon=0.0, punta=4.9, canha_corta=0.0, canha_larga=0.0, arranquio_de_cepa=0.0, estimacion_promedio=90.0, total_perdida=4.9, obs='Solo perdida de punta ', calidad_ref_muestra='9aa11d40fb3d41e1be7b60054ef10c77'),
 muestra(fecha_registro='2023-05-09 20:03:58+00:00', amigo_id='4fd08a7c1cb242b2bed4b5fbdff2cac0', punto='0101000020E61000008BC7FB276E954FC0DDB42AD3794631C0', nombre_muestra='M2', astillas=0.0, tocon=0.0, punta=5.0, canha_corta=0.0, canha_larga=0.0, arranquio_de_cepa=0.0, estimacion_promedio=90.0, total_perdida=5.0, obs='Alta perdida de punta ', calidad_ref_muestra='9aa11d40fb3d41e1be7b60054ef10c77')]

In [38]:
# fotos patos para muestras
fotos = []
for i in muestras:
    query = {'query': 'select muestra.nombre_muestra, gal.s3_filename from dataset_291604 muestra inner join gallery_41829 gal on muestra.amigo_id = gal.source_amigo_id where muestra.amigo_id=\'{amigo_id}\''.format(amigo_id=i.amigo_id)}
    project = 'https://app.amigocloud.com/api/v1/projects/31805/sql'
    select = amigocloud.get(project, query)
    data = select['data'][0]
    fotos.append(data)
    print(data)

{'nombre_muestra': 'M1', 's3_filename': 'IMG_20230509_155825.jpg'}
{'nombre_muestra': 'M2', 's3_filename': '41829_20230509160528645.jpg'}


In [41]:
fotos

[{'nombre_muestra': 'M1', 's3_filename': 'IMG_20230509_155825.jpg'},
 {'nombre_muestra': 'M2', 's3_filename': '41829_20230509160528645.jpg'}]

In [ ]:
doc = DocxTemplate(ruta + "/templates/tpl_word_1ra.docx")

In [42]:
lista_fotos_inline = []
for foto in fotos:
    response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
    file = open(ruta + '/fotos/' + foto['s3_filename'], "wb")
    file.write(response.content)
    file.close()
    lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor=ruta+'/fotos/' + foto['file_name'], width=Mm(120)), 'lote': foto['lote']})

NameError: name 'doc' is not defined